In [92]:
%pip install ragas

Note: you may need to restart the kernel to use updated packages.


In [120]:
import json
import pandas as pd
import pyarrow as pa
from datasets import Dataset

path = 'evalSet.csv'
df = pd.read_csv(path)
df["contexts"] = df["contexts"].apply(json.loads)
df = df.drop(columns=["evolution_type", "episode_done", "generated_contexts", "metadata"])
print(df.head())
dataset = Dataset.from_pandas(df)

                                            question  \
0  What are the different arcane traditions in wi...   
1  What spells can a Tiefling cast as part of the...   
2  What is the feature associated with the keyphr...   
3  What is the effect of the Relentless Rage feat...   
4          What are the class features of a Warlock?   

                                            contexts  \
0  [The most common arcane traditions in the mult...   
1  [***Infernal Legacy.*** You know the *thaumatu...   
2  [### Sorcerous Origin\n\nChoose a sorcerous or...   
3  [### Rage\n\nIn battle, you fight with primal ...   
4  [***Pact Magic.*** If you have both the Spellc...   

                                        ground_truth  \
0  The most common arcane traditions in wizardry ...   
1  The Tiefling can cast the thaumaturgy cantrip,...   
2  The feature associated with the keyphrase "Ele...   
3  Starting at 11th level, the Relentless Rage fe...   
4  As a warlock, you gain the following class 

In [114]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import RunConfig, evaluate
import nest_asyncio
nest_asyncio.apply()


run_config = RunConfig()
run_config.timeout = 120
run_config.thread_timeout = 120
run_config.max_workers = 4

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    run_config=run_config,
    is_async=False,
)

result
resultPandas = result.to_pandas()
display(resultPandas)

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer,generated_contexts,context_precision,faithfulness,answer_relevancy,context_recall
0,What are the different arcane traditions in wi...,[The most common arcane traditions in the mult...,The most common arcane traditions in wizardry ...,simple,"[{'source': 'data/02 classes.md', 'filename': ...",True,"b""The various arcane traditions in wizardry ar...","['\n\nWhen you reach 20th level, you gain mast...",1.000000,1.000000,0.963072,0.285714
1,What spells can a Tiefling cast as part of the...,[***Infernal Legacy.*** You know the *thaumatu...,"The Tiefling can cast the thaumaturgy cantrip,...",simple,"[{'source': 'data/01 races.md', 'filename': 'd...",True,"b""The spells included in a Tiefling's Infernal...",[' in darkness as if it were dim light. You ca...,1.000000,1.000000,0.978651,1.000000
2,What is the feature associated with the keyphr...,[### Sorcerous Origin\n\nChoose a sorcerous or...,"The feature associated with the keyphrase ""Ele...",simple,"[{'source': 'data/02 classes.md', 'filename': ...",True,"b'The feature linked to the keyphrase ""Element...",['\n\nAbility Score Improvement\n\nWhen you re...,0.333333,0.000000,0.990262,0.000000
3,What is the effect of the Relentless Rage feat...,"[### Rage\n\nIn battle, you fight with primal ...","Starting at 11th level, the Relentless Rage fe...",simple,"[{'source': 'data/02 classes.md', 'filename': ...",True,"b""The Relentless Rage feature allows a barbari...",[' feature.\n\nExtra Attack\n\nBeginning at 5t...,0.588889,1.000000,0.939783,1.000000
4,What are the class features of a Warlock?,[***Pact Magic.*** If you have both the Spellc...,"As a warlock, you gain the following class fea...",simple,"[{'source': 'data/02 classes.md', 'filename': ...",True,b'The Warlock class in Dungeons and Dragons is...,['\n\nAbility Score Improvement\n\nWhen you re...,0.950000,0.888889,0.914318,0.066667
5,What are the benefits and limitations of enter...,"[### Rage\n\nIn battle, you fight with primal ...","When a barbarian enters a rage, they gain seve...",simple,"[{'source': 'data/02 classes.md', 'filename': ...",True,"b""Advantages of going into a rage as a barbari...",['Barbarian\n\nClass Features\n\nAs a barbaria...,1.000000,1.000000,0.964891,1.000000
6,What is the ability gained at 14th level that ...,[### Hide in Plain Sight\n\nStarting at 10th l...,Vanish,simple,"[{'source': 'data/02 classes.md', 'filename': ...",True,b'Vanish allows a character to use the Hide ac...,[' no extra movement. You can also pass throug...,0.804167,1.000000,0.896170,1.000000
7,What is the purpose of the Open Game Content i...,[All of the rest of the SRD5 is Open Game Cont...,The purpose of the Open Game Content in the Sy...,simple,"[{'source': 'data/00 legal.md', 'filename': 'd...",True,b'The purpose of Open Game Content in the Syst...,"['Legal Information\n\nPermission to copy, mod...",1.000000,0.750000,0.997591,1.000000
8,What is the relationship between Pact Magic an...,[***Pact Magic.*** If you have both the Spellc...,Pact Magic is a feature of the Warlock class. ...,simple,"[{'source': 'data/02 classes.md', 'filename': ...",True,b'If you have both the Spellcasting class feat...,[' Invocations Known 1st +2 Otherworldly Patro...,0.866667,1.000000,0.871064,1.000000
9,What are some of the Celtic gods and their ass...,"[##### Celtic Deities, | Deity ...",The Celtic gods and their associated domains a...,simple,"[{'source': 'data/13 gods.md', 'filename': 'da...",True,b'Some Celtic gods and their associated areas ...,['Appendix PH-B: Fantasy-Historical Pantheons\...,1.000000,0.500000,0.948148,1.000000


In [115]:
resultPandas.to_csv("result.csv", index=False)
result
# [0.9313194444180828, 0.925, 0.908733879753922, 0.8966666666666667]
# {'context_precision': 0.9383, 'faithfulness': 0.9750, 'answer_relevancy': 0.9098, 'context_recall': 0.9025}
# {'context_precision': 0.9373, 'faithfulness': 0.9750, 'answer_relevancy': 0.9094, 'context_recall': 0.8925}
# {'context_precision': 0.9433, 'faithfulness': 0.9474, 'answer_relevancy': 0.9082, 'context_recall': 0.9025}
# {'context_precision': 0.9401, 'faithfulness': 0.9750, 'answer_relevancy': 0.9120, 'context_recall': 0.9025}
# {'context_precision': 0.9481, 'faithfulness': 0.9150, 'answer_relevancy': 0.9068, 'context_recall': 0.8811}

# {'context_precision': 0.8856, 'faithfulness': 0.8379, 'answer_relevancy': 0.9451, 'context_recall': 0.7946}
# {'context_precision': 0.8942, 'faithfulness': 0.8507, 'answer_relevancy': 0.9427, 'context_recall': 0.7726}


{'context_precision': 0.8942, 'faithfulness': 0.8507, 'answer_relevancy': 0.9427, 'context_recall': 0.7726}